# 在Dynex平台上使用Python进行计算 - BQM

Dynex是全球首个基于DynexSolve芯片算法的神经形态超级计算区块链，采用了一种“有用工作证明”（PoUW）方法来解决现实世界问题。此示例演示了如何使用Dynex SDK在Dynex平台上使用Python进行计算

In [1]:
import dynex
import dimod

## 构建一个二次二元模型

二次二元模型（BQM）是以下形式的问题:

<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>E</mi>
  <mo stretchy="false">(</mo>
  <mrow data-mjx-texclass="ORD">
    <mi mathvariant="bold">v</mi>
  </mrow>
  <mo mathvariant="bold" stretchy="false">)</mo>
  <mo mathvariant="bold">=</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">a</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <munder>
    <mo data-mjx-texclass="OP">&#x2211;</mo>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">&lt;</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </munder>
  <msub>
    <mi mathvariant="bold">b</mi>
    <mrow data-mjx-texclass="ORD">
      <mi mathvariant="bold">i</mi>
      <mo mathvariant="bold">,</mo>
      <mi mathvariant="bold">j</mi>
    </mrow>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">j</mi>
  </msub>
  <mo mathvariant="bold">+</mo>
  <mi mathvariant="bold">c</mi>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <mstyle scriptlevel="0">
    <mspace width="2em"></mspace>
  </mstyle>
  <msub>
    <mi mathvariant="bold">v</mi>
    <mi mathvariant="bold">i</mi>
  </msub>
  <mo>&#x2208;</mo>
  <mo fence="false" stretchy="false">{</mo>
  <mo mathvariant="bold">&#x2212;</mo>
  <mn mathvariant="bold">1</mn>
  <mo mathvariant="bold">,</mo>
  <mo mathvariant="bold">+</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
  <mtext mathvariant="bold">&#xA0;or&#xA0;</mtext>
  <mo fence="false" stretchy="false">{</mo>
  <mn mathvariant="bold">0</mn>
  <mo mathvariant="bold">,</mo>
  <mn mathvariant="bold">1</mn>
  <mo fence="false" stretchy="false">}</mo>
</math>

其中a、b、c为实数值

我们需要定义一个代表我们问题的二次模型（QM）。构建一个二次二元模型（BQM）的最简单方式是使用dimod的符号变量。对于BQM中的每个数学变量，我们使用dimod定义一个符号二元变量。一旦QM被定义，它将被存储为一个BinaryQuadraticModel对象。这个对象存储了数学表达式的线性和二次系数，任何常数项或偏移量，以及用于构建模型的变量类型。在这种情况下，构建的bqmmodel对象会显示以下内容：

In [2]:
# 二元二次模型（BQM）:
bqmodel = dimod.BinaryQuadraticModel({'x1': 1.0, 'x2': -1.5, 'x3': 2.0}, 
                                     {('x1', 'x2'): 1.0, ('x2', 'x3'): -2.0}, 
                                      0.0, dimod.BINARY)

model = dynex.BQM(bqmodel);
print(bqmodel);

BinaryQuadraticModel({'x1': 1.0, 'x2': -1.5, 'x3': 2.0}, {('x2', 'x1'): 1.0, ('x3', 'x2'): -2.0}, 0.0, 'BINARY')


## 与Dynex采样器互动 

为了找到QM的最小能量状态（即为我们的QM提供最小能量值的变量值分配），Dynex SDK提供了采样器和求解器。求解器是运行问题的资源。采样器是多次运行问题以获取样本集合的过程，每个样本都是我们问题的一个可能解决方案。为了方便起见，通常我们会将Dynex的采样器整体称为一个术语，以包括求解器。

In [3]:
sampler = dynex.DynexSampler(model);

一旦我们在程序中建立了我们的采样器，我们可以调用它来对我们的QM进行采样。每种类型的QM模型都有其与采样器交互的方法，无论是QUBO、BinaryQuadraticModel还是其他任何QM。我们使用Dynex的采样函数之一来调用采样器来对我们的QM进行采样，具体取决于我们使用的QM类型。例如，下面的代码片段演示了如何使用Dynex平台对名为bqm的BinaryQuadraticModel对象进行采样。

In [20]:
sampleset = sampler.sample(num_reads=32, annealing_time = 10);

time: 0.01s #workers: 7 #chips: 224 #steps: 11 global loc: 0 global energy: 0.370625
FINISHED READ AFTER 0.01 SECONDS
SAMPLESET LOADED


在我们对QM进行采样后，采样器会返回一个SampleSet对象。该对象包含所有返回的样本及其相应的能量值、芯片数量、积分步数等信息。附加信息的内容取决于使用的采样器。随着用户对Dynex SDK和各种可用的采样器变得更加熟悉，通常有必要花一些时间来探索SampleSet对象中提供的丰富信息。我们可以访问的SampleSet的一些关键属性和方法如下：

In [21]:
print(sampleset[-1]) # sample with the lowest energy

{'sample': ['0.258750', '-1.000000', '1.000000'], 'chips': 224, 'steps': 11, 'loc': 0, 'energy': 0.370625}


样本显示了我们QM的变量X1、X2和X3的相应能量值，以及总能量、芯片数量和积分步数等附加信息。样本显示了我们QM的变量X1、X2和X3的相应能量值，以及总能量、芯片数量和积分步数等附加信息。

In [22]:
sampleset # 完整的样本集合

[{'sample': ['0.258750', '-1.000000', '1.000000'],
  'chips': 32,
  'steps': 2,
  'loc': 0,
  'energy': 0.370625},
 {'sample': ['1.000000', '-1.000000', '1.000000'],
  'chips': 32,
  'steps': 1,
  'loc': 0,
  'energy': 1.5},
 {'sample': ['-0.000000', '-0.431250', '1.000000'],
  'chips': 32,
  'steps': 2,
  'loc': 0,
  'energy': 1.353125},
 {'sample': ['1.000000', '-1.000000', '1.000000'],
  'chips': 32,
  'steps': 1,
  'loc': 0,
  'energy': 0.5},
 {'sample': ['0.258750', '-1.000000', '0.345000'],
  'chips': 32,
  'steps': 2,
  'loc': 0,
  'energy': 1.025625},
 {'sample': ['1.000000', '-0.431250', '1.000000'],
  'chips': 32,
  'steps': 2,
  'loc': 0,
  'energy': 0.853125},
 {'sample': ['1.000000', '-1.000000', '0.000000'],
  'chips': 32,
  'steps': 1,
  'loc': 0,
  'energy': 1.0},
 {'sample': ['0.258750', '-1.000000', '1.000000'],
  'chips': 224,
  'steps': 11,
  'loc': 0,
  'energy': 0.370625}]

每行显示了一个返回的样本（解决方案），以及相应的能量值、使用的芯片数量（每个Dynex芯片提供一次读取），和积分步数（=退火时间）。请注意，有效的积分步数可以低于指定的退火时间，因为ODE积分在找到解决方案时结束。